<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/ode_new_with_mse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ode solver için gerekli kütüphaneler
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

# Deep learning için gerekli kütüphaneler 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

%matplotlib inline
"""
def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]
"""
tf.random.set_seed(39)
np.random.seed(39)

initial_state = [0., 1.]                  # t0 anındaki başlangıç değerleri
t_starting_and_last = [0, 2*np.pi]        # Fonksiyonun tanımlı olacağı alanın başlangıç ve bitiş noktasını tanımlıyoruz.
t_eval = np.arange(0, 2*np.pi, 0.4)      # t_starting_and_last ile belirlediğimiz aralıkta hesaplanacak t değerlerini bu şekilde girebiliriz, hiçbirşey girilmez ise default olarak kendisi parçalıyor.

def F(t,y):       # Diff eq F(t,s) = 0
  return [y[1], -y[0]]

sol = solve_ivp(fun= F, t_span= t_starting_and_last, y0= initial_state, t_eval= t_eval, dense_output=True)


plt.figure(figsize = (8, 8))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (4, 4))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()


In [ ]:
y = sol.y.T # etiketlerimiz
t = sol.t   # t değerlerimiz

In [ ]:
t = t.reshape(([t.shape[0],1]))
shuffle = np.concatenate((y,t), axis=1)
np.random.shuffle(shuffle)
shuffle

In [ ]:
y = shuffle[:,:2]
t = shuffle[:,2]
print('y:',y, '\nt:',t)

In [ ]:
def H(z):
  return tf.reduce_sum((z**2)/2,axis=1)

mse = tf.keras.losses.MSE
def custom_loss(y_true, y_pred):
  return mse(y_true, y_pred) + (h_0 - H(y_pred))**2

# Hamiltonian for initial values [0,1]
h_0 = H(np.array([initial_state], dtype=np.float32))

In [ ]:
# Model
girdi = Input(shape=(1,))

hidden = Dense(64, activation= tf.keras.activations.tanh)(girdi)
hidden = Dense(64, activation= tf.keras.activations.tanh)(hidden)

hidden = Dense(2)(hidden)

model = Model(inputs=girdi, outputs=hidden)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss= custom_loss)
model.fit(x=t, y= y, epochs=500, batch_size=64)

In [ ]:
t_test = np.arange(0, 2*np.pi, 0.02).reshape((315,1)) #629
pred = model.predict(t_test)

hamiltonian = H(pred)
true_hamiltonian = H(sol.y.T)

plt.figure(figsize = (8, 4))

plt.subplot(121)
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.plot(pred[:, 0], pred[:, 1])
plt.xlabel('q')
plt.ylabel('p')

plt.subplot(122)
plt.plot(t_eval, true_hamiltonian)
plt.plot(t_test,hamiltonian)
plt.xlabel('t')
plt.ylabel('H')

plt.tight_layout()
plt.show()

In [ ]:
hamiltonian

In [ ]:
array = np.array([0.50265914, 0.5020327 , 0.501523  , 0.50111926, 0.5008098 ,
       0.50058216, 0.50042665, 0.5003306 , 0.5002844 , 0.5002767 ,
       0.50029945, 0.500344  , 0.5004028 , 0.50047046, 0.5005407 ,
       0.5006101 , 0.5006752 , 0.500735  , 0.5007869 , 0.50083077,
       0.50086683, 0.50089544, 0.50091684, 0.500934  , 0.50094616,
       0.5009557 , 0.5009633 , 0.5009706 , 0.5009775 , 0.50098544,
       0.50099474, 0.50100565, 0.5010174 , 0.50103015, 0.5010431 ,
       0.50105566, 0.50106734, 0.50107723, 0.5010842 , 0.50108784,
       0.5010879 , 0.5010824 , 0.50107217, 0.5010563 , 0.5010355 ,
       0.5010088 , 0.50097734, 0.5009414 , 0.5009015 , 0.5008579 ,
       0.5008124 , 0.5007656 , 0.50071776, 0.5006703 , 0.5006239 ,
       0.5005794 , 0.50053805, 0.5004992 , 0.5004645 , 0.50043297,
       0.5004064 , 0.5003832 , 0.50036407, 0.50034875, 0.50033724,
       0.5003283 , 0.50032204, 0.5003177 , 0.5003152 , 0.500313  ,
       0.5003119 , 0.50031066, 0.5003095 , 0.5003074 , 0.50030464,
       0.5003009 , 0.5002964 , 0.50029075, 0.50028455, 0.5002775 ,
       0.50027   , 0.5002623 , 0.50025433, 0.50024724, 0.5002403 ,
       0.50023425, 0.5002298 , 0.50022626, 0.50022465, 0.5002244 ,
       0.5002261 , 0.50023025, 0.5002354 , 0.5002431 , 0.50025153,
       0.5002621 , 0.50027376, 0.50028604, 0.50029904, 0.5003121 ,
       0.5003252 , 0.5003382 , 0.50035   , 0.50036067, 0.50037   ,
       0.500377  , 0.50038254, 0.50038576, 0.5003856 , 0.5003834 ,
       0.5003782 , 0.50037026, 0.5003594 , 0.50034547, 0.50032914,
       0.50031036, 0.50028884, 0.5002657 , 0.5002402 , 0.5002139 ,
       0.500186  , 0.5001575 , 0.500128  , 0.5000988 , 0.50007004,
       0.50004196, 0.50001454, 0.49998862, 0.49996474, 0.49994236,
       0.49992234, 0.4999046 , 0.49988934, 0.49987698, 0.49986714,
       0.4998603 , 0.49985576, 0.49985418, 0.49985474, 0.49985808,
       0.4998637 , 0.49987063, 0.4998797 , 0.49989054, 0.49990213,
       0.49991515, 0.49992865, 0.4999425 , 0.49995637, 0.4999699 ,
       0.49998316, 0.49999538, 0.50000685, 0.5000171 , 0.5000264 ,
       0.5000342 , 0.50004053, 0.5000456 , 0.5000487 , 0.5000505 ,
       0.5000511 , 0.50005   , 0.5000479 , 0.5000441 , 0.5000399 ,
       0.50003463, 0.50002885, 0.5000221 , 0.50001514, 0.50000834,
       0.5000012 , 0.49999455, 0.49998784, 0.49998143, 0.49997553,
       0.49997032, 0.49996552, 0.49996156, 0.49995816, 0.4999551 ,
       0.4999527 , 0.4999507 , 0.49994922, 0.49994794, 0.49994677,
       0.49994603, 0.49994498, 0.49994385, 0.49994248, 0.49994063,
       0.49993858, 0.49993584, 0.49993265, 0.4999288 , 0.49992454,
       0.4999189 , 0.49991322, 0.499907  , 0.49989983, 0.49989215,
       0.4998839 , 0.49987572, 0.49986678, 0.49985796, 0.49984902,
       0.49984005, 0.49983132, 0.49982315, 0.4998152 , 0.499808  ,
       0.4998015 , 0.4997964 , 0.49979192, 0.49978906, 0.49978745,
       0.4997878 , 0.49978954, 0.49979308, 0.4997983 , 0.49980605,
       0.49981555, 0.49982718, 0.49984065, 0.49985597, 0.4998732 ,
       0.49989164, 0.49991155, 0.49993226, 0.49995372, 0.49997467,
       0.49999574, 0.5000155 , 0.5000333 , 0.5000491 , 0.50006187,
       0.5000712 , 0.50007653, 0.50007635, 0.5000714 , 0.50006026,
       0.500043  , 0.50001895, 0.49998814, 0.49995074, 0.4999062 ,
       0.49985495, 0.49979788, 0.4997345 , 0.49966598, 0.49959323,
       0.49951696, 0.4994382 , 0.49935818, 0.49927828, 0.4991999 ,
       0.4991247 , 0.4990543 , 0.49899003, 0.4989338 , 0.4988877 ,
       0.49885285, 0.49883106, 0.4988235 , 0.49883193, 0.49885678,
       0.4988987 , 0.49895808, 0.4990349 , 0.49912816, 0.4992367 ,
       0.499359  , 0.49949282, 0.4996354 , 0.4997834 , 0.49993363,
       0.5000815 , 0.50022316, 0.5003544 , 0.50047106, 0.5005687 ,
       0.5006431 , 0.50069165, 0.5007112 , 0.5007001 , 0.50065607,
       0.5005796 , 0.50047034, 0.5003303 , 0.5001613 , 0.49996716,
       0.4997514 , 0.4995194 , 0.49927673, 0.49902904, 0.49878326,
       0.49854594, 0.49832404, 0.49812445, 0.4979539 , 0.4978194 ,
       0.49772596, 0.49767986, 0.49768645, 0.4977498 , 0.4978731 ,
       0.49806055, 0.49831444, 0.49863622, 0.49902743, 0.49948904,
       0.50002074, 0.50062275, 0.5012946 , 0.50203514, 0.5028433 ])
plt.figure(figsize = (4, 4))
plt.plot(t_test,array,c='red')
plt.plot(t_test, hamiltonian, c='green')
plt.plot(t_test, [0.5 for i in range(t_test.shape[0])])
plt.plot(t_eval, true_hamiltonian)

plt.xlabel('t')
plt.ylabel('H')
plt.show()